In [1]:
import numpy as np
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard

In [2]:
file = open("baddate.txt", encoding="utf-8")
text = file.read()

print(text[0:300])

THE BAD DATE CHRONICLES
Written by
Rick Garman
Based on a story by
Jennifer Notas
PRODUCTION WHITE: 3/21/16
BLUE TBD
Bad Date Productions
2400 Boundary Road
Burnaby, BC, V5M 3Z3
Ph: 604.292.5260
Fax: 604.628.3001
1 1 EXT. PORTLAND - NIGHT
Establishing shot of Portland, Oregon. *
2 2 EXT. RICHARDS RE


In [9]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

seqlen = 40
step = seqlen
sentences = []
for i in range(40, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])
    

x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1
        
        
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    for diversity in [0.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            
            
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])
          
#model.fit(x, y,
          #batch_size=128,
          #epochs=150,
          #callbacks=[print_callback])

C:\Users\syosh\AppData\Local\Temp\ipykernel_23696\1048562009.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
C:\Users\syosh\AppData\Local\Temp\ipykernel_23696\1048562009.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), seqlen, len(chars)), dtype

Epoch 1/10
22/22 [==============================] - ETA: 0s - loss: 3.5813 - categorical_crossentropy: 3.5813 - accuracy: 0.1403
----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: "ew.
It’ll be huge!
QUICK CUT TO:
47 47 I"
ew.
It’ll be huge!
QUICK CUT TO:
47 47 Iree a i  ital  e as t lt e m  e e ila arn i aare ie a t  e t aneee a
 o a 
oe
ce et 
 we  ea h es u e    al  hhe t e  ahee h a he  i iaIshareti    e    re
e li e  t a l  n oH r u t e h    aor  t o.
n st ane  a i l lo tu e. he a  o. au   at t ngir 
 I
22/22 [==============================] - 40s 2s/step - loss: 3.5813 - categorical_crossentropy: 3.5813 - accuracy: 0.1403are . t nthanewatore ae  ee
Epoch 2/10
22/22 [==============================] - ETA: 0s - loss: 2.9567 - categorical_crossentropy: 2.9567 - accuracy: 0.2396
----- Generating text after Epoch: 1
----- diversity: 0.5
----- Generating with seed: "e you
don’t want me to take you home?
LE"
e you
don’t want me to take you home?
LEIGH


KeyboardInterrupt: 

In [ ]:
file2 = open("nietzsche.txt", encoding="utf-8")
text2 = file2.read()

In [ ]:
chars = sorted(list(set(text2)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

seqlen = 40
step = seqlen
sentences = []
for i in range(40, len(text2) - seqlen - 1, step):
    sentences.append(text2[i: i + seqlen + 1])
    

x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1
        
        
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text2) - seqlen - 1)
    
    for diversity in [0.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text2[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            preds = model.predict(x_pred, verbose=0)
            next_index = sample(preds[0, -1], diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            
            
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=120,
          epochs=10,
          callbacks=[print_callback])
          
#model.fit(x, y,
          #batch_size=128,
          #epochs=150,
          #callbacks=[print_callback])